In [ ]:
import os, sys
from dotenv import load_dotenv

from telethon import TelegramClient, errors
import asyncio
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download("stopwords")
sw = stopwords.words("russian")

load_dotenv()

api_id = os.getenv('APP_ID_TG')  # Ваш api_id для аккаунта пользователя
api_hash = os.getenv('API_HASH_TG')  # Ваш api_hash для аккаунта пользователя
channels = [
    "@tass_agency",
    "@rian_ru",
    "@kommersant",
    "@gazeta_ru",
    "@meduzalive",
    "@rbc_news"
]  # Список каналов

In [19]:
def clean_text_ru(text): 
   # Приведение текста к нижнему регистру
    text = text.lower()
    # Замена всех не-словесных символов на пробел (кроме букв и знаков препинания)
    text = re.sub(r'\W+', ' ', text)

    # Удаление URL-адресов
    text = re.sub(r"http\S+", "", text)

    # Создание шаблона для HTML-тегов
    html = re.compile(r'&lt;.*?&gt;')

    # Удаление HTML-тегов из текста
    text = html.sub(r'', text)

    # Список пунктуаций для удаления
    punctuations = '@#!?+&amp;*[]-%.:/();$=&gt;&lt;|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p, '')  # Удаление пунктуации

    # Удаление стоп-слов и приведение слов к нижнему регистру
    text = [word.lower() for word in text.split() if word.lower() not in sw]

    # Объединение слов обратно в текст
    text = " ".join(text)

    # Создание шаблона для поиска эмодзи
    emoji_pattern = re.compile("["
                        u"\U0001F600-\U0001F64F"  # эмоции
                        u"\U0001F300-\U0001F5FF"  # символы и пиктограммы
                        u"\U0001F680-\U0001F6FF"  # транспорт и карты
                        u"\U0001F1E0-\U0001F1FF"  # флаги
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        "]+", flags=re.UNICODE)

    # Удаление эмодзи из текста
    text = emoji_pattern.sub(r'', text)

    return text

In [ ]:
import re

client = TelegramClient("session_news", api_id, api_hash)


async def load_telegram_news(channel, limit=100):
    """
    Загружает последние сообщения из указанного Telegram-канала.
    """
    try:
        print(f"Подключение к Telegram для обработки канала {channel}...")
        await client.start()

        docs = []
        print(f"Загрузка сообщений из канала {channel}...")
        async for msg in client.iter_messages(channel, limit=limit):
            if not msg.text:
                continue

            # Очистка текста
            cleaned_text = clean_text_ru(msg.text)
            print(f"Оригинальный текст: {msg.text[:50]}...")  # Отладка
            print(f"Очищенный текст: {cleaned_text[:50]}...")  # Отладка

            docs.append({
                "id": msg.id,
                "channel": channel,
                "text": cleaned_text,
                "date": msg.date.isoformat()
            })
        print(f"Загрузка из канала {channel} завершена. Получено {len(docs)} сообщений.")
        return docs
    except errors.UsernameInvalidError:
        print(f"Канал {channel} недоступен или не существует.")
        return []
    except errors.FloodWaitError as e:
        print(f"Превышен лимит запросов для канала {channel}. Ожидание {e.seconds} секунд.")
        await asyncio.sleep(e.seconds)
        return []
    except Exception as e:
        print(f"Ошибка при обработке канала {channel}: {e}")
        return []
    finally:
        print(f"Отключение от Telegram после обработки канала {channel}...")
        await client.disconnect()


def prepare_rag_documents(messages):
    """
    Преобразует сообщения в формат, подходящий для RAG.
    """
    print("Подготовка данных для RAG...")
    print(f"Количество сообщений для подготовки: {len(messages)}")  # Отладка
    rag_docs = [
        {
            "id": msg["id"],
            "content": msg["text"],
            "metadata": {
                "source": "telegram",
                "channel": msg["channel"],
                "date": msg["date"]
            }
        }
        for msg in messages
    ]
    print(f"Подготовлено {len(rag_docs)} документов.")
    return rag_docs


async def main():
    all_messages = []
    for channel in channels:
        print(f"Обработка канала {channel}...")
        messages = await load_telegram_news(channel, limit=100)
        print(f"Сообщений из канала {channel}: {len(messages)}")  # Отладка
        all_messages.extend(messages)

    if all_messages:
        print("Подготовка новостей для RAG...")
        rag_docs = prepare_rag_documents(all_messages)
        print("Пример подготовленного документа:")
        print(rag_docs[0] if rag_docs else "Нет документов для отображения.")  # Отладка
        return all_messages, rag_docs
    else:
        print("Нет доступных сообщений.")
        return [], []


# Запуск main() и получение данных
all_messages, rag_docs = await main()
print(f"Всего сообщений: {len(all_messages)}")  # Отладка
print(f"Всего документов для RAG: {len(rag_docs)}")  # Отладка

Обработка канала @tass_agency...
Подключение к Telegram для обработки канала @tass_agency...
Загрузка сообщений из канала @tass_agency...
Загрузка сообщений из канала @tass_agency...
Ошибка при обработке канала @tass_agency: name 're' is not defined
Отключение от Telegram после обработки канала @tass_agency...
Сообщений из канала @tass_agency: 0
Обработка канала @rian_ru...
Подключение к Telegram для обработки канала @rian_ru...
Ошибка при обработке канала @tass_agency: name 're' is not defined
Отключение от Telegram после обработки канала @tass_agency...
Сообщений из канала @tass_agency: 0
Обработка канала @rian_ru...
Подключение к Telegram для обработки канала @rian_ru...
Загрузка сообщений из канала @rian_ru...
Загрузка сообщений из канала @rian_ru...
Ошибка при обработке канала @rian_ru: name 're' is not defined
Отключение от Telegram после обработки канала @rian_ru...
Сообщений из канала @rian_ru: 0
Обработка канала @kommersant...
Подключение к Telegram для обработки канала @komme

In [21]:
rag_docs

[]

In [14]:
1

1